# Context-sensitive Spelling Correction

The goal of the assignment is to implement context-sensitive spelling correction. The input of the code will be a set of text lines and the output will be the same lines with spelling mistakes fixed.

Submit the solution of the assignment to Moodle as a link to your GitHub repository containing this notebook.

Useful links:
- [Norvig's solution](https://norvig.com/spell-correct.html)
- [Norvig's dataset](https://norvig.com/big.txt)
- [Ngrams data](https://www.ngrams.info/download_coca.asp)

Grading:
- 60 points - Implement spelling correction
- 20 points - Justify your decisions
- 20 points - Evaluate on a test set


## Implement context-sensitive spelling correction

Your task is to implement context-sensitive spelling corrector using N-gram language model. The idea is to compute conditional probabilities of possible correction options. For example, the phrase "dking sport" should be fixed as "doing sport" not "dying sport", while "dking species" -- as "dying species".

The best way to start is to analyze [Norvig's solution](https://norvig.com/spell-correct.html) and [N-gram Language Models](https://web.stanford.edu/~jurafsky/slp3/3.pdf).

When solving this task, we expect you'll face (and successfully deal with) some problems or make up the ideas of the model improvement. Some of them are: 

- solving a problem of n-grams frequencies storing for a large corpus;
- taking into account keyboard layout and associated misspellings;
- efficiency improvement to make the solution faster;
- ...

Please don't forget to describe such cases, and what you decided to do with them, in the Justification section.

##### IMPORTANT:  
Your project should not be a mere code copy-paste from somewhere. You must provide:
- Your implementation
- Analysis of why the implemented approach is suggested
- Improvements of the original approach that you have chosen to implement

In [2]:
import re
from collections import Counter
from tqdm import tqdm

def words(text):
    return re.findall(r'\w+', text.lower())

def pairs(text):
    return zip(text, text[1:])

# Read the file once.
with open('./text.txt', encoding='utf-8') as f:
    text_content = f.read()

word_list = words(text_content)
WORDS = Counter(word_list)
PAIRS = Counter(pairs(word_list))

# Filter out words with counts <= 150.
WORDS = Counter({
    word: count 
    for word, count in tqdm(WORDS.items(), desc="Filtering WORDS")
    if count > 150
})

# Filter PAIRS such that both words in the pair exist in the filtered WORDS.
PAIRS = Counter({
    pair: count 
    for pair, count in tqdm(PAIRS.items(), desc="Filtering PAIRS")
    if pair[0] in WORDS and pair[1] in WORDS
})


Filtering PAIRS: 100%|██████████| 24280903/24280903 [00:10<00:00, 2262028.41it/s]


In [ ]:
WORDS_SMOOTHING_VALUE = 1 / (sum(WORDS.values()) + 1)
PAIRS_SMOOTHING_VALUE = 1 / (sum(PAIRS.values()) + 1)


In [ ]:
class SpellCorrector:

    def P(self, word):
        "Probability of `word`."
        return WORDS[word] / sum(WORDS.values()) if WORDS[word] > 0 else WORDS_SMOOTHING_VALUE

    def candidates(self, word):
        "Generate possible spelling corrections for `word`."
        return (self.known([word]) or self.known(self.edits1(word)) or self.known(self.edits2(word)) or [word])

    def known(self, words):
        "The subset of `words` that appear in the dictionary of WORDS."
        return set(w for w in words if w in WORDS)

    def edits1(self, word):
        "Create a set of possible edits that are one edit away from `word`."
        letters     = 'abcdefghijklmnopqrstuvwxyz'
        splits      = [(word[:i], word[i:]) for i in range(len(word) + 1)]
        deletes     = [L + R[1:] for L, R in splits if R]
        transposes  = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R) > 1]
        replaces    = [L + c + R[1:] for L, R in splits for c in letters if R]
        inserts     = [L + c + R for L, R in splits for c in letters]
        return set(deletes + transposes + replaces + inserts)

    def edits2(self, word):
        "Create a set of possible edits that are two edits away from `word`."
        return (e2 for e1 in self.edits1(word) for e2 in self.edits1(e1))

    def bigram_prob(self, w1, w2):
        "Return the probability of w2 following w1, using BIGRAMS counts and WORDS."
        if w1 is None:
            return 1
        
        return PAIRS[w1, w2] / WORDS[w1] if PAIRS[w1, w2] > 0 else PAIRS_SMOOTHING_VALUE

    def correction(self, sentence):
        """
        Perform global, bidirectional context-sensitive correction.
        Computes forward (left-to-right) and backward (right-to-left) DP tables
        using bigram probabilities, then combines them to choose the best candidate.
        """
        
        tokens = sentence.split()
        n = len(tokens)
        
        # For each token, get candidate corrections.
        candidates_list = [([word] if word in WORDS else list(self.candidates(word))) for word in tokens]
        
        # Forward DP pass.
        dp_forward = [{} for _ in range(n)]
        # Initialize first token.
        for cand in candidates_list[0]:
            dp_forward[0][cand] = (self.P(cand), None)  # (score, backpointer)
        
        # Fill forward table.
        for i in range(1, n):
            for cand in candidates_list[i]:
                best_prev = None
                best_score = 0
                for prev_cand, (prev_score, _) in dp_forward[i - 1].items():
                    score = prev_score * self.bigram_prob(prev_cand, cand)
                    if score > best_score:
                        best_score = score
                        best_prev = prev_cand
                dp_forward[i][cand] = (best_score * self.P(cand), best_prev)
        
        # Backward DP pass.
        dp_backward = [{} for _ in range(n)]
        # Initialize last token.
        for cand in candidates_list[-1]:
            dp_backward[-1][cand] = (self.P(cand), None)
        
        # Fill backward table.
        for i in range(n - 2, -1, -1):
            for cand in candidates_list[i]:
                best_next = None
                best_score = 0
                for next_cand, (next_score, _) in dp_backward[i + 1].items():
                    score = self.bigram_prob(cand, next_cand) * next_score
                    if score > best_score:
                        best_score = score
                        best_next = next_cand
                dp_backward[i][cand] = (self.P(cand) * best_score, best_next)
        
        # Combine forward and backward scores.
        # For each position choose the candidate that maximizes the product
        # of forward and backward scores.
        best_sequence = []
        for i in range(n):
            best_cand = max(
                candidates_list[i],
                key=lambda cand: dp_forward[i][cand][0] *
                                dp_backward[i][cand][0]
            )
            best_sequence.append(best_cand)
        
        return " ".join(best_sequence)

# Example usage:
sentence = "ths is a smple sentnce"
correction = SpellCorrector().correction
corrected = correction(sentence)
print("Corrected Sentence:", corrected)


Corrected Sentence: this is a simple sentence


## Justify your decisions

Write down justificaitons for your implementation choices. For example, these choices could be:
- Which ngram dataset to use
- Which weights to assign for edit1, edit2 or absent words probabilities
- Beam search parameters
- etc.

### Justification of the approach

#### Overview:
The implemented spell checker uses a context-sensitive approach based on a bidirectional N-gram language model. This method leverages both left and right contexts to improve the accuracy of spelling corrections, especially in sentences where the context significantly influences the correct word choice.

#### Key Components and Decisions:

1. **N-gram Language Model**:
   - **Bigram Model**: The model uses bigrams to capture the relationship between consecutive words. This helps in understanding the context better than a unigram model, which considers words independently.
   - **Bidirectional Correction**: The correction algorithm performs two passes—one from left to right (forward) and one from right to left (backward). This bidirectional approach ensures that the context from both sides of a word is considered, leading to more accurate corrections.

2. **Probability Calculations**:
   - **Unigram Probability**: The probability of a word occurring independently is calculated based on its frequency in the corpus.
   - **Bigram Probability**: The probability of a word given its preceding word is calculated using bigram counts. This helps in determining the likelihood of word sequences.
   - **Smoothing**: Smoothing is applied to handle unseen bigrams, ensuring that the model can still make reasonable predictions even for rare or unseen word pairs.

3. **Dynamic Programming (Viterbi Algorithm)**:
   - **Forward Pass**: In the forward pass, the algorithm computes the best sequence of corrections from the beginning to the end of the sentence.
   - **Backward Pass**: In the backward pass, the algorithm computes the best sequence of corrections from the end to the beginning of the sentence.
   - **Combination of Scores**: The final correction is determined by combining the scores from both the forward and backward passes, selecting the candidate that maximizes the combined score.

4. **Candidate Generation**:
   - **Known Words**: The algorithm first checks if the word is already known (exists in the vocabulary).
   - **One-Edit Distance**: If the word is not known, the algorithm generates candidates that are one edit away (insertions, deletions, substitutions, and transpositions).
   - **Two-Edit Distance**: If no known candidates are found within one edit distance, the algorithm generates candidates that are two edits away.



#### The dataset:

For the assignment large unified text corpus was used. The texts that were used to create the corpus are:
- [eng_news_2015_1M-sentences.txt](https://wortschatz.uni-leipzig.de/en/download/English)
- [eng_news_2018_1M-sentences.txt](https://wortschatz.uni-leipzig.de/en/download/English)
- [eng_news_2019_1M-sentences.txt](https://wortschatz.uni-leipzig.de/en/download/English)
- [eng_news_2023_1M-sentences.txt](https://wortschatz.uni-leipzig.de/en/download/English)
- [eng_news_2024_1M-sentences.txt](https://wortschatz.uni-leipzig.de/en/download/English)
- [eng_wikipedia_2010_1M-sentences](https://wortschatz.uni-leipzig.de/en/download/English)
- [eng_wikipedia_2012_1M-sentences](https://wortschatz.uni-leipzig.de/en/download/English)
- [eng_wikipedia_2016_1M-sentences](https://wortschatz.uni-leipzig.de/en/download/English)
- [eng-simple_wikipedia_2021_300K-sentences](https://wortschatz.uni-leipzig.de/en/download/English)

#### Probabilities:
- If the word is unknown, the probability of its correction is $\frac{1}{|V|}$, where $|V|$ is the total number of words in the vocabulary.
- If the word is known, the probability of its independent correction is $P(w) = \frac{C(w)}{|V|}$, where $C(w)$ is the count of the word in the corpus and $|V|$ is the total number of words in the vocabulary.
- If the word is known, the probability of its left-dependent correction is $P(w|w_{i-1}) = P(w) * \frac{C(w_{i-1}, w)}{C(w_{i-1})}$, where $C(w, w_{i-1})$ is the count of the bigram $(w_{i-1}, w)$ in the corpus and $C(w_{i-1})$ is the count of the word $w_{i-1}$ in the corpus.
- If the word is known, the probability of its right-dependent correction is $P(w|w_{i+1}) = P(w) * \frac{C(w, w_{i+1})}{C(w_{i+1})}$, where $C(w, w_{i+1})$ is the count of the bigram $(w, w_{i+1})$ in the corpus and $C(w_{i+1})$ is the count of the word $w_{i+1}$ in the corpus.
- If the bigram is unknown, the probability of its correction is $\frac{1}{|B|}$, where $|B|$ is the total number of bigrams.
- The bidirectional probability of the bigram is $P(w_{i-1}, w_{i+1}|w) = P(w|w_{i-1}) * P(w|w_{i+1})$.





## Evaluate on a test set

Your task is to generate a test set and evaluate your work. You may vary the noise probability to generate different datasets with varying compexity (or just take another dataset). Compare your solution to the Norvig's corrector, and report the accuracies.

In [5]:
class NorvigSpellCorrector:
    def P(self, word, N=sum(WORDS.values())): 
        "Probability of `word`."
        return WORDS[word] / N

    def correction(self, word): 
        "Most probable spelling correction for word."
        return max(self.candidates(word), key=self.P)

    def candidates(self, word): 
        "Generate possible spelling corrections for word."
        return (self.known([word]) or self.known(self.edits1(word)) or self.known(self.edits2(word)) or [word])

    def known(self, words): 
        "The subset of `words` that appear in the dictionary of WORDS."
        return set(w for w in words if w in WORDS)

    def edits1(self, word):
        "All edits that are one edit away from `word`."
        letters    = 'abcdefghijklmnopqrstuvwxyz'
        splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
        deletes    = [L + R[1:]               for L, R in splits if R]
        transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
        replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
        inserts    = [L + c + R               for L, R in splits for c in letters]
        return set(deletes + transposes + replaces + inserts)

    def edits2(self, word): 
        "All edits that are two edits away from `word`."
        return (e2 for e1 in self.edits1(word) for e2 in self.edits1(e1))


In [10]:
import pandas as pd

test_sentences = [
    "dking sport",
    "the qick brown fox",
    "speling corector iz fyn",
    "species are dking",
    "he is a great playr",
    "my frend is doing well",
    "what a wondrful wrold",
    "the sun is shning",
    "the weeather is nicee",
    "i loe programing",
    "ths iy a tst",
    "helto worl",
    "spellng is fun",
]

test_words = [
    "dking",
    "speling",
    "qick",
    "playr",
    "wondrful",
    "frend",
]

norvig = NorvigSpellCorrector().correction
enhanced_norvig = SpellCorrector().correction

# Collect results for sentences
sentence_results = []
for sentence in test_sentences:
    norvig_correction = norvig(sentence)
    enhanced_correction = enhanced_norvig(sentence)
    sentence_results.append([sentence, norvig_correction, enhanced_correction])

# Collect results for words
word_results = []
for word in test_words:
    norvig_correction = norvig(word)
    enhanced_correction = enhanced_norvig(word)
    word_results.append([word, norvig_correction, enhanced_correction])

# Create DataFrames
sentence_df = pd.DataFrame(sentence_results, columns=['Original Sentence', 'Norvig Correction', 'Enhanced Correction'])
word_df = pd.DataFrame(word_results, columns=['Original Word', 'Norvig Correction', 'Enhanced Correction'])

# Display results
print("Sentence Corrections:")
print(sentence_df.to_string(index=False, col_space=30))

print("\nWord Corrections:")
print(word_df.to_string(index=False, col_space=30))


Sentence Corrections:
             Original Sentence              Norvig Correction            Enhanced Correction
                   dking sport                    dking sport                    doing sport
            the qick brown fox             the qick brown fox            the quick brown fox
       speling corector iz fyn        speling corector iz fyn      spelling corrector is fun
             species are dking              species are dking              species are doing
           he is a great playr            he is a great playr           he is a great player
        my frend is doing well         my frend is doing well        my friend is doing well
         what a wondrful wrold          what a wondrful wrold         what a wonderful world
             the sun is shning              the sun is shning             the sun is shining
         the weeather is nicee          the weeather is nicee            the weather is nice
              i loe programing               i l

In [7]:
nsc = NorvigSpellCorrector()

def unit_tests(corrector):
    assert corrector.correction('speling') == 'spelling'              # insert
    assert corrector.correction('korrectud') == 'corrected'           # replace 2
    assert corrector.correction('bycycle') == 'bicycle'               # replace
    assert corrector.correction('inconvient') == 'inconvenient'       # insert 2
    assert corrector.correction('arrainged') == 'arranged'            # delete
    assert corrector.correction('peotry') =='poetry'                  # transpose
    assert corrector.correction('peotryy') =='poetry'                 # transpose + delete
    assert corrector.correction('word') == 'word'                     # known
    return f'unit_tests for {corrector} pass'

unit_tests(NorvigSpellCorrector()), unit_tests(SpellCorrector())


('unit_tests for <__main__.NorvigSpellCorrector object at 0x000002218D260340> pass',
 'unit_tests for <__main__.SpellCorrector object at 0x000002218D260340> pass')

#### Comparison of Norvig’s Spell Corrector vs. Context-Sensitive Spell Corrector:

| Aspect                         | Norvig’s Spell Corrector                          | Enhanced Spell Corrector                                 |
|--------------------------------|---------------------------------------------------|----------------------------------------------------------|
| **Context Handling**           | Uses unigram probabilities and candidate generation based on one- and two-edit distances. This works well for isolated words. | Incorporates context through bigram probabilities as well as bidirectional (forward and backward) dynamic programming. It leverages both left- and right-contexts, leading to more informed corrections. |
| **Candidate Generation**       | Generates candidates from known words, one-edit, and two-edits; correction is selected based solely on word frequency. | Uses the same candidate generation strategy but weighs candidate scores by combining their unigram probabilities with the likelihoods of word pairs (bigram probabilities) from neighboring context. |
| **Sentence Level Correction**  | Applies correction word by word; often leaves multi-word errors unresolved. | Corrects entire sentences using a global optimization (Viterbi algorithm) that considers the sequence of corrections, allowing corrections that depend on surrounding words (e.g., “dking sport” &rarr; “doing sport”). |
| **Errors Resolved**            | Performs well on simple misspellings, but may struggle with context-dependent errors. | More likely to select context-appropriate corrections, especially for homophones or common word confusions where the surrounding context is important (e.g., “qick” &rarr; “quick” when surrounded by “the” and “brown fox”). |
| **Test Results**               | Norvig’s approach left “dking sport” unchanged and “speling corector iz fyn” unchanged. | The enhanced version corrected “dking sport” to “doing sport” and transformed “speling corector iz fyn” into “spelling corrector is fun”, demonstrating its sensitivity to context. |

#### Advantages of the Enhanced Approach:

- **Context Sensitivity:**  
  By combining forward and backward bigram probabilities, the enhanced model chooses corrections that fit naturally within the sentence context.

- **Bidirectional Processing:**  
  The use of dynamic programming in both directions ensures the selection of a candidate that is consistent with its immediate neighbors, reducing the risk of isolated correction errors.

- **Improved Correction Accuracy:**  
  The approach rectifies errors that require context (e.g., “dking species” vs. “dking sport”), thus producing more natural and semantically correct sentence outputs.

Overall, while Norvig’s solution provides a solid baseline based on word frequencies and minimal edit distances, the enhanced approach offers significant improvements when dealing with context-dependent errors by taking advantage of both left and right word relationships.

#### Useful resources (also included in the archive in moodle):

1. [Possible dataset with N-grams](https://www.ngrams.info/download_coca.asp)
2. [Damerau–Levenshtein distance](https://en.wikipedia.org/wiki/Damerau–Levenshtein_distance#:~:text=Informally%2C%20the%20Damerau–Levenshtein%20distance,one%20word%20into%20the%20other.)